# Telco Customer Churn: Comprehensive Model Evaluation

This notebook provides a deep dive into model evaluation, focusing on metrics and techniques that are crucial for imbalanced classification problems like churn prediction.

## 1. Setup and Model Training

In [ ]:
import pandas as pd
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import seaborn as sns
import yaml

# Add src directory to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Data and Preprocessing
from src.data_loader import TelcoDataLoader
from src.preprocessor import DataPreprocessor
from sklearn.model_selection import train_test_split

# Models
from src.base_model import LogisticRegressionModel
from src.ensemble_models import RandomForestChurnModel

# Evaluation
from src.model_evaluator import ModelEvaluator
from src.business_evaluator import BusinessEvaluator
from sklearn.metrics import precision_recall_curve, auc

# Load config
with open('../config/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Load and preprocess data
loader = TelcoDataLoader()
df = loader.load_raw_data()
df_processed = DataPreprocessor.preprocess_data(df.copy())

# Create features and target
X = df_processed.drop(config['target'], axis=1)
y = df_processed[config['target']].apply(lambda x: 1 if x == 'Yes' else 0)

# Create and apply preprocessing pipeline
preprocessor = DataPreprocessor().create_preprocessing_pipeline()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=config['training']['random_state'], stratify=y)
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Train models
lr = LogisticRegressionModel(random_state=config['training']['random_state'])
lr.train(X_train_transformed, y_train)
rf = RandomForestChurnModel(random_state=config['training']['random_state'])
rf.train(X_train_transformed, y_train)

models_dict = {'Logistic Regression': lr.model, 'Random Forest': rf.model}

## 2. Standard Performance Metrics

In [ ]:
evaluator = ModelEvaluator(models_dict, X_test_transformed, y_test)
comparison_df = evaluator.compare_multiple_models()
print("--- Model Comparison (Standard Metrics) ---")
print(comparison_df)

While accuracy might look good, it can be misleading in imbalanced datasets. ROC AUC is a better metric, but let's dive deeper.

## 3. Confusion Matrices

In [ ]:
# The plots are saved to the reports/figures/visualization directory
evaluator.create_evaluation_plots()
print('Confusion matrices and ROC curves have been saved to the reports/figures/visualization directory.')

## 4. Precision-Recall Curves

Precision-Recall curves are often more informative than ROC curves for imbalanced datasets, as they focus on the performance of the positive (minority) class.

In [ ]:
plt.figure(figsize=(10, 8))

for name, model in models_dict.items():
    y_pred_proba = model.predict_proba(X_test_transformed)[:, 1]
    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    pr_auc = auc(recall, precision)
    plt.plot(recall, precision, label=f'{name} (PR AUC = {pr_auc:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curves')
plt.legend(loc='lower left')
plt.show()

## 5. Cost-Sensitive Evaluation

In [ ]:
# Using the imblearn classification report which provides metrics like geometric mean
evaluator.evaluate_imbalanced_classification('Random Forest')

## 6. Business Impact Analysis

In [ ]:
# --- Business Assumptions ---
AVG_MONTHLY_REVENUE = 70.0
AVG_CUSTOMER_LIFESPAN_MONTHS = 24
CUSTOMER_ACQUISITION_COST = 300.0
RETENTION_CAMPAIGN_COST_PER_CUSTOMER = 50.0
CAMPAIGN_SUCCESS_RATE = 0.4 # Assume our campaign can convince 40% of targeted churners to stay

business_eval = BusinessEvaluator(
    avg_monthly_revenue=AVG_MONTHLY_REVENUE,
    avg_customer_lifespan_months=AVG_CUSTOMER_LIFESPAN_MONTHS,
    customer_acquisition_cost=CUSTOMER_ACQUISITION_COST,
    retention_campaign_cost_per_customer=RETENTION_CAMPAIGN_COST_PER_CUSTOMER
)

# Generate report for the Random Forest model
y_pred_rf = rf.predict(X_test_transformed)
business_eval.generate_business_impact_report(y_test, y_pred_rf, campaign_success_rate=CAMPAIGN_SUCCESS_RATE)